## Fontes de Dados

### Our World in Data ([fonte](https://ourworldindata.org/co2-emissions))

- Emissão de CO<sub>2</sub> em milhões de toneladas (2015 para Cuba, Coreia do Norte e Palestina, 2016 par países restantes)
- PIB em dólares ajustdo para inflação de 2011 (2015 para Cuba, Coreia do Norte e Palestina, 2016 par países restantes)
- População (2015 para Cuba, Coreia do Norte e Palestina, 2016 par países restantes)

### ONU ([fonte](http://hdr.undp.org/en/indicators))

- **Índice de gini de distribuição de renda familiar (anos variados)**
- IDH (2019)
- Porcentagem da população residindo em áreas urbanas (2019)
- Porcentagem da variação na área de floresta (diferença entre 1990 e 2016)
- **Porcentagem da energia consumida proveniente de combustíveis fósseis (anos variados)**
- Porcentagem do Rendimento Nacional Bruto derivado da extração de recursos naturais (anos variados)
- Índice de desigualdade de gênero (2019)
- Expectativa de vida no nascimento (2019)
- **Porcentagem da população vivendo abaixo da linha da pobreza (anos variados)**
- **Porcentagem do PIB investido em pesquisa e desenvolvimento (anos variados)**
- **Proporção dos gastos públicos em educação e saúde sobre gastos militares (anos variados)**
- Porcentagem de importações e exportações sobre o PIB (anos variados)
- **Índice de desemprego (2019)**

### Center for Systemic Peace ([fonte](https://www.systemicpeace.org/polityproject.html))

- Polity Score, índice que avalia o nível de democracia de cada país - removemos países com valores especiais para evitar interferência na regressão (2018)

### Repórteres sem Fronteira ([fonte](https://rsf.org/en/ranking))

- Índice de liberdade de imprensa (2021)

**Dados em negrito estão disponíveis para menos de 90% dos países selecionados**

In [1]:
import json
import csv

data = {}

co2_file_path = "owid-co2-data.json"
with open(co2_file_path, 'r') as json_file:
    co2_data = json.load(json_file)
    for key in co2_data.keys():
        if 'iso_code' in co2_data[key] and key != 'World':
            for i in range(len(co2_data[key]['data'])):
                if 'co2' in co2_data[key]['data'][i] and 'gdp' in co2_data[key]['data'][i] \
                        and 'population' in co2_data[key]['data'][i]:
                    data[co2_data[key]['iso_code']] = {
                        'name': key,
                        'co2_emissions': co2_data[key]['data'][i]['co2'],
                        'gdp': co2_data[key]['data'][i]['gdp'],
                        'population': co2_data[key]['data'][i]['population']               
                    }

X_fields = ['gdp', 'population']
                   
def get_iso3_from_country_name(country_name):
    iso3 = [key for key, values in data.items() if values['name'] == country_name]
    return iso3[0] if iso3 else None

def add_csv_data(file, country_row, data_row, data_name):
    X_fields.append(data_name)
    with open(file) as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        co2_names = [(data[key]['name'], key) for key in data]
        for line in reader:
            country_key = get_iso3_from_country_name(line[country_row])
            if country_key:
                data[country_key][data_name] = float(line[data_row])

def check_missing_countries(file, country_row):
    with open(file) as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        co2_names = [data_dict[key]['name'] for key in data_dict]
        file_names = [line[country_row] for line in reader]
        return [name for name in co2_names if name not in file_names]

add_csv_data('gini-index.csv', 1, -2, 'gini_index')
add_csv_data('hdi.csv', 1, -2, 'hdi')
add_csv_data('urban-population.csv', 1, -2, 'urban_population')
add_csv_data('forest-area-change.csv', 1, -2, 'forest_area_change')
add_csv_data('fossil-fuel-percentage.csv', 1, -2, 'fossil_fuel_percentage')
add_csv_data('natural-resource-depletion.csv', 1, -2, 'natural_resource_depletion')
add_csv_data('gender-inequality-index.csv', 1, -2, 'gender_inequality_index')
add_csv_data('life-expectancy.csv', 1, -2, 'life_expectancy')
add_csv_data('poverty-line.csv', 1, -2, 'poverty_line')
add_csv_data('research-and-development.csv', 1, -2, 'research_and_development')
add_csv_data('education-health-military-expenses.csv', 1, -2, 'education_health_military_expenses')
add_csv_data('exports-imports.csv', 1, -2, 'exports_imports')
add_csv_data('unemployment.csv', 1, -2, 'unemployment')
add_csv_data('democracy.csv', 4, 10, 'democracy')
add_csv_data('press-freedom.csv', 3, 7, 'press_freedom')

print('Países:', len(data.keys()))

Países: 164


In [2]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

countries = []
X = []
y = []
for values in data.values():
    countries.append(values['name'])
    y.append(values['co2_emissions'])
    X.append([values[field] if field in values else np.nan for field in X_fields])

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X = imp.fit_transform(X)
y = np.array(y)

X_train, X_test, y_train, y_test, countries_train, countries_test \
        = train_test_split(X, y, countries, test_size=0.1, random_state=0)

print("Training countries:", countries_train)
print("Testing countries:", countries_test)

Training countries: ['Niger', 'Myanmar', 'Zambia', 'North Macedonia', 'Sudan', 'Costa Rica', 'Germany', 'Togo', 'Azerbaijan', 'Turkmenistan', 'Georgia', 'Tanzania', 'Sao Tome and Principe', 'Guinea-Bissau', 'Haiti', 'United Kingdom', 'Canada', 'Taiwan', 'Slovakia', 'Bulgaria', 'Mexico', 'South Africa', 'Bosnia and Herzegovina', 'Honduras', 'North Korea', 'Djibouti', 'India', 'Cambodia', 'Yemen', 'Japan', 'Rwanda', 'Laos', 'Saint Lucia', 'Chad', 'Montenegro', 'Russia', 'Tunisia', 'Hong Kong', 'Nigeria', 'Brazil', 'Israel', 'Colombia', 'Libya', 'Jamaica', 'Oman', 'Algeria', 'Guinea', 'Lithuania', 'Turkey', 'Ecuador', 'Bangladesh', 'Malta', 'Uzbekistan', 'Mozambique', 'Mauritania', 'Lesotho', 'Sierra Leone', 'Uruguay', 'Finland', 'Portugal', 'Trinidad and Tobago', 'Hungary', 'Norway', 'Pakistan', 'Iraq', 'Ethiopia', 'Eswatini', 'Belgium', 'Romania', 'Cameroon', 'Philippines', 'Burkina Faso', 'Bolivia', 'Kuwait', 'Gabon', 'Kazakhstan', 'Angola', 'Zimbabwe', 'Mongolia', 'Australia', 'Iran',